In [19]:
from octis.dataset.dataset import Dataset
import hdbscan
import sys
sys.path.append('C:\\Users\\Martin\\Documents\\GitHub\\master\\evaluation')
import trainer
import helper
import pandas as pd

In [2]:
from sentence_transformers import SentenceTransformer

In [3]:
import multimodal
import c_tf_idf
import multimodalModel

In [4]:
output_folder = './results/experiment_1'
dataset_folder = './multimodal_custom_octis_dataset/'

In [5]:
mm_dataset = Dataset()
mm_dataset.load_custom_dataset_from_folder(dataset_folder)

In [6]:
embedding_model = SentenceTransformer("clip-ViT-B-32")

c:\Users\Martin\AppData\Local\Programs\Python\Python38\lib\site-packages\torchvision\transforms\transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [24]:

path_to_data = './multimodal_data.tsv'

try:
    texts, images = multimodal.get_image_and_text_from_file(path_to_data)
except FileNotFoundError:
    base_path = '..\\data\\processed'
    ends = ['processed_part-00000-48a6f07e-bb86-4735-aac7-883349f41a28-c000.tsv'
            ,'processed_part-00001-48a6f07e-bb86-4735-aac7-883349f41a28-c000.tsv'
            ,'processed_part-00002-48a6f07e-bb86-4735-aac7-883349f41a28-c000.tsv'
            ,'processed_part-00003-48a6f07e-bb86-4735-aac7-883349f41a28-c000.tsv'
            ,'processed_part-00004-48a6f07e-bb86-4735-aac7-883349f41a28-c000.tsv'
            ,]
    paths = [f'{base_path}\\{end}' for end in ends]
    df = helper.load_all_data(paths)
    df.to_csv(path_to_data, index=False, sep='\t')
    texts, images = helper.get_image_and_text_from_df(df)


In [25]:
text_embed = multimodal.get_embeddings_from_text(texts, embedding_model)
image_embed = multimodal.get_embeddings_from_images(images, embedding_model)

 37%|███▋      | 34/91 [00:33<00:55,  1.03it/s]c:\Users\Martin\AppData\Local\Programs\Python\Python38\lib\site-packages\PIL\Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 91/91 [01:37<00:00,  1.07s/it]


In [10]:
metricsloader = trainer.MetricsLoader(dataset_folder)

In [27]:
params = {'path_to_data':path_to_data,
'embedding_model':SentenceTransformer("clip-ViT-B-32"),

'precomputed_text_embeds':text_embed,
'precomputed_image_embeds':image_embed
}
base_name = 'experiment_1_mm_'

min_cluster_sizes = [100, 200, 400]

get_extra_info = True
fitted_clusterers = []
image_representatives = []
top_terms_representatives = []
for min_cluster_size in min_cluster_sizes:
    clusterer = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size)
    params['clusterer']=clusterer

    mm_trainer = trainer.Trainer(custom_dataset_folder=dataset_folder, params=params, metrics_loader=metricsloader)
    helper.run_experiment(mm_trainer, output_folder, name=f'{base_name}unrestricted_topics_min_cluster_size_{min_cluster_size}')
    if get_extra_info:
        last_fit_clusterer = mm_trainer.model.clusterer
        fitted_clusterers.append(last_fit_clusterer)

        mm_trainer.model.find_image_representatives()
        last_fit_images = mm_trainer.model.cluster_images
        last_fit_terms = mm_trainer.model.top_terms

        image_representatives.append(last_fit_images)
        top_terms_representatives.append(last_fit_terms)

c:\Users\Martin\AppData\Local\Programs\Python\Python38\lib\site-packages\torchvision\transforms\transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


start:14032023_14_00_31


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Martin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 48919/48919 [00:00<00:00, 182505.25it/s]


npmi:-0.03206677800424157
diversity:0.7666666666666667
wetc:0.6887939178750458


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Martin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 48919/48919 [00:00<00:00, 212391.42it/s]


npmi:-0.021224415696315158
diversity:0.7545454545454545
wetc:0.6712302803245654


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Martin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 48919/48919 [00:00<00:00, 194384.83it/s]


npmi:-0.021337860427075993
diversity:0.8222222222222222
wetc:0.6492234325461014
end:14032023_14_00_31


AttributeError: module 'trainer' has no attribute 'model'

In [ ]:
for fc in fitted_clusterers:
    fc.condensed_tree_.plot(select_clusters=True)

In [ ]:
params = {'path_to_data':path_to_data,
'embedding_model':SentenceTransformer("clip-ViT-B-32"),

'precomputed_text_embeds':text_embed,
'precomputed_image_embeds':image_embed,
'n_clusters':10
}
mm_trainer = trainer.Trainer(custom_dataset_folder=dataset_folder, params=params, metrics_loader=metricsloader)
helper.run_experiment(mm_trainer, output_folder, name=f'{base_name}restricted_topics')